In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../')

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from personflow.loader import PersonalityPostLoader
from personflow.preparator import PersonContainer, TrainTestSplitter

In [11]:
loader = PersonalityPostLoader()
persons = loader.run()

container = PersonContainer(persons)
container.split_posts(10)
#container.replace_personality_codes()
#container.replace_links()
#container.replace_digits()
container.evenly_distribute()
container.post_joiner()

Ignored 5624 texts because of too small chunks


In [12]:
ttsplitter = TrainTestSplitter(container)
train_x, train_y, test_x, test_y = ttsplitter.run()

Train size: Counter({'Introvert': 6728, 'Extrovert': 6727})
Test size: Counter({'Extrovert': 2884, 'Introvert': 2883})


In [13]:
train_x[1]

"steve job's was recognized for his striving for efficiency and practicality. his genius is in his systemization of inventions, less so than in invention. this is where claims of se and te come from. pencil. not inf fe. type ~= emotion, type == what drove emotion obvious troll but either way, he'd get there by use of his ni or te. you can get any behavior with any cognition, all cognition describes is the thought process, which may be prone to develop certain behaviors... every edgy ntp and their mother acts on their off days. the very idea of 'borrowing' types and progressively chameleoning screams ne and fe, specifically in that order. it's cause you spend all your time irl ignoring your need for expression, so now you're excitedly exaggerating and projecting your weirdness because you get aroused off the image of you it puts in... just gonna ignore all your rules.  si - normal se - fighting ni - psychic ne - electric ti - steel te - dark fi - grass fe - fairy ixfj  must research si 

In [28]:
tokenizer = Tokenizer(num_words=50000, oov_token="$oov")
tokenizer.fit_on_texts(train_x)
word_index = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(train_x)
train_seq_padded = pad_sequences(
    sequences, maxlen=300, padding='post', truncating='post')

test_seq = tokenizer.texts_to_sequences(test_x)
test_seq_padded = pad_sequences(test_seq, maxlen=300)

In [29]:
print(train_seq_padded[1])

[ 1  1 29  1 15  1  1 15  1  5  1  1  1 10 12  1  1  7  1  1 27 95 12  1
 19 10  1  1  7  1  5  1  1 57  1 23  1  1 90  1 90 34  1  1  1  1 14  1
 83  1 54 65 76  1  7  1  1 33  1  8 44 54  1  1 17  1  1 42  1  1 10  3
  1  1  1  1 20  1  4  1  1  1  1  1  1  5  1  1  1 25  1  1  1  3 62  1
  7  1  1  5  1  1  1  1  5  1  1 12  9  1 41  1  8  1 42 38 61  1  1 38
  1 15  1 27 97  1  1  1  5  1 38  1 55  8 54  1  1  3  1  7  8 11  1 12
 31  1  1 42 38  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  2  1 43  1  1 67  2 35 41 72  1 12  6 48  1  1 23  1 85 39
 24  4 20 12  3  1  5 41  1  1  1  3 90  7  1 51 22 27 39  1  4  1  1  1
  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0]
